In [5]:
#!pip install ortools

In [6]:
from ortools.sat.python import cp_model

# 创建模型
model = cp_model.CpModel()

##  `model.AddNoOverlap`



- `interval` 区间不重叠
    - 区间集合，一系列区间集合

In [8]:
# 定义任务的持续时间
duration_task1 = 2
duration_task2 = 3

# 创建两个任务
task1_start = model.NewIntVar(0, 100, 'start_task1')
task1_end = model.NewIntVar(0, 100, 'end_task1')
task1_interval = model.NewIntervalVar(task1_start, duration_task1, task1_end, 'interval_task1')

task2_start = model.NewIntVar(0, 100, 'start_task2')
task2_end = model.NewIntVar(0, 100, 'end_task2')
task2_interval = model.NewIntervalVar(task2_start, duration_task2, task2_end, 'interval_task2')

In [9]:
# 添加NoOverlap约束
model.AddNoOverlap([task1_interval, task2_interval])

# 求解模型
solver = cp_model.CpSolver()
status = solver.Solve(model)

# 打印解决方案
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution:")
    print("Task1 starts at", solver.Value(task1_start))
    print("Task1 ends at", solver.Value(task1_end))
    print("Task2 starts at", solver.Value(task2_start))
    print("Task2 ends at", solver.Value(task2_end))
else:
    print("No solution found")

Solution:
Task1 starts at 0
Task1 ends at 2
Task2 starts at 2
Task2 ends at 5


In [10]:
# 添加NoOverlap约束
model.AddNoOverlap([task1_interval, task2_interval])
# 任务2在任务1前
model.Add(task2_end <= task1_start)
# 求解模型
solver = cp_model.CpSolver()
status = solver.Solve(model)

# 打印解决方案
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution:")
    print("Task1 starts at", solver.Value(task1_start))
    print("Task1 ends at", solver.Value(task1_end))
    print("Task2 starts at", solver.Value(task2_start))
    print("Task2 ends at", solver.Value(task2_end))
else:
    print("No solution found")

Solution:
Task1 starts at 3
Task1 ends at 5
Task2 starts at 0
Task2 ends at 3


## packing

In [1]:
from ortools.sat.python import cp_model

# Instance
container = (40, 15)
boxes = [
    (11, 3),
    (13, 3),
    (9,  2),
    (7,  2),
    (9,  3),
    (7,  3),
    (11, 2),
    (13, 2),
    (11, 4),
    (13, 4),
    (3,  5),
    (11, 2),
    (2,  2),
    (11, 3),
    (2,  3),
    (5,  4),
    (6,  4),
    (12, 2),
    (1,  2),
    (3,  5),
    (13, 5),
    (12, 4),
    (1,  4),
    (5,  2),
    #(6,  2),  # add to make tight
    # (6,3), # add to make infeasible
]
model = cp_model.CpModel()

# We have to create the variable for the bottom left corner of the boxes.
# We directly limit their range, such that the boxes are inside the container
x_vars = [model.NewIntVar(0, container[0]-box[0], name = f'x1_{i}') for i, box in enumerate(boxes)]
y_vars = [model.NewIntVar(0, container[1]-box[1], name = f'y1_{i}') for i, box in enumerate(boxes)]
# Interval variables are actually more like constraint containers, that are then passed to the no overlap constraint
# Note that we could also make size and end variables, but we don't need them here
x_interval_vars = [model.NewIntervalVar(start=x_vars[i], size=box[0], end=x_vars[i]+box[0], name = f'x_interval_{i}') for i, box in enumerate(boxes)]
y_interval_vars = [model.NewIntervalVar(start=y_vars[i], size=box[1], end=y_vars[i]+box[1], name = f'y_interval_{i}') for i, box in enumerate(boxes)]
# Enforce that no two rectangles overlap
model.AddNoOverlap2D(x_interval_vars, y_interval_vars)

# Solve!
solver = cp_model.CpSolver()
solver.parameters.log_search_progress = True
solver.log_callback = print
status = solver.Solve(model)
assert status == cp_model.OPTIMAL
for i, box in enumerate(boxes):
    print(f'box {i} is placed at ({solver.Value(x_vars[i])}, {solver.Value(y_vars[i])})')


Starting CP-SAT solver v9.7.2996

Starting CP-SAT solver v9.7.2996
Parameters: log_search_progress: true
Setting number of workers to 64
Parameters: log_search_progress: true
Setting number of workers to 64


Initial satisfaction model '': (model_fingerprint: 0x932179154af0ad79)
#Variables: 48
  - 3 in [0,10]
  - 6 in [0,11]
  - 6 in [0,12]
  - 9 in [0,13]
  - 4 in [0,27]
  - 2 in [0,28]
  - 5 in [0,29]
  - 2 in [0,31]
  - 2 in [0,33]
  - 1 in [0,34]
  - 2 in [0,35]
  - 2 in [0,37]
  - 2 in [0,38]
  - 2 in [0,39]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Starting presolve at 0.00s
Initial satisfaction model '': (model_fingerprint: 0x932179154af0ad79)
#Variables: 48
  - 3 in [0,10]
  - 6 in [0,11]
  - 6 in [0,12]
  - 9 in [0,13]
  - 4 in [0,27]
  - 2 in [0,28]
  - 5 in [0,29]
  - 2 in [0,31]
  - 2 in [0,33]
  - 1 in [0,34]
  - 2 in [0,35]
  - 2 in [0,37]
  - 2 in [0,38]
  - 2 in [0,39]
#kInterval: 48
#kLinear1: 48
#kNoOverlap2D: 1 (#rectangles: 24)

Starting pres